## Inital Data Upload and EDA

In this notebook, we will upload and begin to explore our dataset(s). First, let's set up our standard modules.

In [3]:
# Import standard data science & visualization packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Our first dataset comes from [Moneypuck.com](moneypuck.com). It contains Advanced Stats for all NHL games from the 2008-09 season to the 2022-23 season, aggregated at a team level.

In [4]:
mp_df = pd.read_csv('all_teams.csv')

In [13]:
mp_df.shape

(190300, 111)

In [9]:
# .info won't show the individual column details unless we include the number of columns to show. There are 111 columns in this dataset.
mp_df.info(111)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190300 entries, 0 to 190299
Data columns (total 111 columns):
 #    Column                                     Dtype  
---   ------                                     -----  
 0    team                                       object 
 1    season                                     int64  
 2    name                                       object 
 3    gameId                                     int64  
 4    playerTeam                                 object 
 5    opposingTeam                               object 
 6    home_or_away                               object 
 7    gameDate                                   int64  
 8    position                                   object 
 9    situation                                  object 
 10   xGoalsPercentage                           float64
 11   corsiPercentage                            float64
 12   fenwickPercentage                          float64
 13   iceTime                    

In [10]:
mp_df.head()

,team,season,name,gameId,playerTeam,opposingTeam,home_or_away,gameDate,position,situation,...,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,playoffGame
0,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,other,...,1.0,1.000,0.0,0.017,0.000,0.000,0.037,0.037,0.037,0
1,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,all,...,31.0,30.369,5.0,0.396,0.168,0.168,2.917,2.833,2.714,0
2,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,5on5,...,20.0,19.369,3.0,0.237,0.168,0.168,1.862,1.777,1.665,0
3,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,4on5,...,9.0,9.000,1.0,0.124,0.000,0.000,0.795,0.795,0.789,0
4,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,5on4,...,1.0,1.000,1.0,0.019,0.000,0.000,0.224,0.224,0.224,0


Each row in the dataframe represents team stats for a specific game. We can see that there are columns indicating the team, as well as the opposing team they faced in that game. There is also a unique gameID column that follows a standard format to identify each game separately. Each gameID has two separate sets of rows, one set for each teHowever, there are multiple rows for each team/game combination.